Do not merge cells. Each cell need to be executed on their own and wait for UI to update the diagram.

In [ ]:
# if you see a timeout error, please restart the kernel and run again.
import cx_fbl.cx_lib as cxlib
import itertools
import networkx as nx
cx = cxlib.CX_Constructor(nm[0], 'Givon_frontiers17')

In [ ]:
# These can be performed by clicking on corresponding items, too.
cx.disable_neurons(['PB/R3/s-EB/3/b-lal/LDG/b', 'PB/L1/s-FB/(2,R[2-3])/s-LAL/RHB/b'])
# cx.disable_synapses([...])
# cx.disable_subregions([...])

In [ ]:
# remove disabled components from the database query
res = cx.remove_components()

In [ ]:
shape = (200, 500)
dt = 1e-4
dur = 0.2
steps = int(dur/dt)
video_type = 'moving_bar_r2l'
bar_width = 50
BU_video_config = {'type': video_type,
                   'bar_width': bar_width,
                   'record': 'video.h5',
                   'record_interval': 100 #record every 100 steps of input video
                  }
BU_rf_config = {'sigma': 0.05}
bu_video_config = {'type': video_type,
                'bar_width': bar_width,
                'record': None}
bu_rf_config = {'sigma': 0.05}
PB_video_config = {'type': video_type,
                   'bar_width': bar_width,
                   'record': None}
PB_rf_config = {'num_glomeruli': 18}

BU_rid = cx.rid_dict['Neuropil']['BU']
BU_models = [cx.find_models(neuron)[0] for neuron in cx.owns(BU_rid, 'Neuron')]
BU_neurons = {rid: cx.data.node[rid] for rid in BU_models}

bu_rid = cx.rid_dict['Neuropil']['bu']
bu_models = [cx.find_models(neuron)[0] for neuron in cx.owns(bu_rid, 'Neuron')]
bu_neurons = {rid: cx.data.node[rid] for rid in bu_models}

PB_subregions = ['PB/L{}'.format(i) for i in range(1,10)] + \
                ['PB/R{}'.format(i) for i in range(1,10)]
PB_rids = itertools.chain.from_iterable([cx.neuron_uid_by_family('PB-EB-LAL', k) for k in PB_subregions])
PB_neurons = {rid: cx.data.node[rid] for rid in PB_rids}

# Since the PB-EB-LAL neuron is a PB neuron, it will be executed in the PB LPU.
# So we first create an entry for PB in the input_processors dict,
# with name "PB_InputProcessor" (See cx_fbl.cx_input).
# All the parameters should be spelt out in the dict.
# Additional input processors can be specified by extending the list.
# We do the same for 'BU' and 'bu' with "BU_InputProcessor".
input_processors = {'PB': [{'class': 'PB_InputProcessor',
                            'name': 'PB',
                            'module': 'cx_fbl.cx_input', # this is where PB_InputProcessor should be imported from
                            'shape': shape,
                            'dt': dt,
                            'dur': dur,
                            'video_config': PB_video_config,
                            'rf_config': PB_rf_config,
                            'neurons': PB_neurons,
                            'record_file': 'PB_input.h5',
                            'record_interval': 10}],
                    'BU': [{'class': 'BU_InputProcessor', 
                            'name': 'BUr',
                            'module': 'cx_fbl.cx_input', # this is where BU_InputProcessor should be imported from
                            'shape': shape,
                            'dt': dt,
                            'dur': dur,
                            'video_config': BU_video_config,
                            'rf_config': BU_rf_config,
                            'neurons': BU_neurons}],
                    'bu': [{'class': 'BU_InputProcessor',
                            'name': 'bul',
                            'module': 'cx_fbl.cx_input',
                            'shape': shape,
                            'dt': dt,
                            'dur': dur,
                            'video_config': bu_video_config,
                            'rf_config': bu_rf_config,
                            'neurons': bu_neurons}]}

# Specify an output processor that is to record all nodes with an output of 'V' or 'spike_state'.
# The sample interval for voltage is every 10 steps. Spikes will be returned with spike time.
output_processors = {'PB': [{'class': 'Record',
                             'uid_dict': {'V': {'uids': None, 'sample_interval': 10},
                                          'spike_state': {'uids': None}}}
                           ]
                    }




In [ ]:
cx.execute(input_processors = input_processors,
           output_processors = output_processors,
           steps = steps, dt = dt)

In [ ]:
# get result from execution. Must wait until you receive back data from the last step
result = cx.get_result()

In [ ]:
# Create a video with video input and all PB_EB_LAL spike outputs
%matplotlib inline
from IPython.display import HTML
anim = cx.animate_PB_EB_LAL()
HTML(anim.to_html5_video())

In [ ]:
# Plot PB_EB_LAL spikes:
cx.plot_PB_EB_LAL(order = 'PB')